In [1]:
import spotipy
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import re

load_dotenv('../secrets/.env')

%run useful_functions.ipynb
%run spotify_scraping.ipynb

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [2]:
#shows = get_shows()
#show_broadcast_details = get_show_broadcast_details()
#broadcast_details = get_broadcast_details()

#master_df = (
#broadcast_playlists
#.merge(broadcast_details, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
#.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
#.merge(shows, on='Show Key', how='left')
#.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
#)
#demo_df = master_df#[(master_df['Service Date'].apply(lambda date: date.year) == 2022) & (master_df['Show Name'] == "Radio 1's Indie Show with Jack Saunders")]

In [3]:
broadcast_playlists = get_broadcast_playlists()
tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()

tracks_to_run = broadcast_playlists[['Track Name', 'Artist Name']].sort_values(['Artist Name','Track Name'], ascending =  True).drop_duplicates(subset=['Artist Name','Track Name'], keep='first')
tracks_to_run = tracks_to_run.merge(tracks[['Original Search Artist','Original Search Track']], how='left', left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], copy=False,indicator=True)
tracks_to_run = tracks_to_run[tracks_to_run['_merge']!="both"][['Artist Name','Track Name']]
tracks_to_run

,Artist Name,Track Name
0,,Faking Jazz Together
1,+HIRS+,Y'all Are Precious.
2,...From The Benthic Zone,Edwin
14,5 Billion in Diamonds,Gravity Rules
17,801,Tomorrow Never Knows
...,...,...
10345,†††,Goodbye Horses
10346,†††,Initiation
10347,†††,Protection
10349,†††,Vivien


In [4]:
for i, (index, row) in enumerate(tracks_to_run.iterrows()):
    print("Iteration ",i+1," of ",len(tracks_to_run.index),":",row['Artist Name'],":", row['Track Name'])
    (temp_tracks,temp_albums,temp_artists) = get_track_info(row['Artist Name'],row['Track Name'])
    
    tracks = pd.concat([tracks,temp_tracks])
    albums = pd.concat([albums,temp_albums])
    artists = pd.concat([artists,temp_artists])
    
tracks = tracks.drop_duplicates(subset=['Original Search Artist', 'Original Search Track'], keep='last')
albums = albums.drop_duplicates(subset='Album ID', keep='last')
artists = artists.drop_duplicates(subset='Artist ID', keep='last')

tracks.to_pickle("../data/spotify_tracks.pkl")
albums.to_pickle("../data/spotify_albums.pkl")
artists.to_pickle("../data/spotify_artists.pkl")

Iteration  1  of  3857 :  : Faking Jazz Together


In [51]:
tracks.to_pickle("../data/spotify_tracks.pkl")
albums.to_pickle("../data/spotify_albums.pkl")
artists.to_pickle("../data/spotify_artists.pkl")

In [53]:
temp = tracks[['Track Name', 'Original Search Track', 'Original Search Artist']].applymap(lambda s: s.lower() if type(s) == str else s)
temp = temp[temp['Track Name'] != temp['Original Search Track']]
temp
#temp.to_excel('./temp.xlsx')
#NEED TO INVESTIGATE PROBLEMS TO IMPROVE MATCHING, see temp excel file eddited

,Track Name,Original Search Track,Original Search Artist
0,hertz,hertz (glastonbury 2022),amyl and the sniffers
0,there’d better be a mirrorball,there'd better be a mirrorball,arctic monkeys
0,there’d better be a mirrorball,there'd better be a mirrorball (instrumental),arctic monkeys
0,i’m sorry,i'm sorry,artemas
0,i’m a mess (with yungblud),i'm a mess,avril lavigne
...,...,...,...
0,we'll make this ok (with travis barker),we'll make this ok,renforshort
0,feel away (feat. james blake & mount kimbie),feel away,slowthai
0,low down - pola & bryson remix,low down (pola & bryson remix),venbee
0,jaws,jaws (instrumental),yunè pinku
